<a href="https://colab.research.google.com/github/denisecammarota/icgpu/blob/main/Clase1_HelloWorld_Sumas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%html
<marquee style='width: 100%; color: blue;'><b>ICNPG2021 en Google Colaboratory-Instituto Balseiro-Clase 1 </b></marquee>


# Preparativos para programar CUDA C/C++ en google colabs

Hola!, Bueno, aqui va un ejemplito de como correr codigo CUDA C/C++ en colabs
[[1]](https://https://www.wikihow.com/Run-CUDA-C-or-C%2B%2B-on-Jupyter-(Google-Colab).

No se olviden de Runtime-> Change Runtime Type -> GPU. Para que funque a cada linea la tienen que ejecutar con un Shift-Enter o Ctrl-Enter o el botoncito de play.

miremos que version de nvcc tenemos

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


A ver que placa nos toco...

In [ ]:
!nvidia-smi

Fri Apr  9 20:23:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

lindas GPUs!!. Ahora, para poder correr Cuda C/C++ instalamos un plugin

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git


  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-kqk3ix62
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-kqk3ix62
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=bd7c4ebd004c5427030e05bc4366593d4ada1c95737c3aa7795fa057d1db7b20
  Stored in directory: /tmp/pip-ephem-wheel-cache-bg5v9iz6/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


y luego:

In [ ]:
%load_ext nvcc_plugin


directory /content/src already exists
Out bin /content/result.out


Listo!, con eso ya podemos correr codigo CUDA C/C++ en el notebook Jupyter.

Para terminar, es conveniente montar nuestro google drive

# ¡Hola Mundo en GPU!

Probemos ahora un programita simple. Notar que al principio tiene que tener un %%cu

In [ ]:
%%cu
#include <stdlib.h>
#include <stdio.h>

__global__ void helloFromGPU()
{
    printf("Hello World from GPU!\n");
}

int main(int argc, char **argv)
{
    printf("Hello World from CPU!\n");

    helloFromGPU<<<1, 10>>>(); 
    cudaDeviceSynchronize();
    return 0;
}

Hello World from CPU!



- probamos modificando la cantidad de hilos, cambia cantidad de veces de impresión (o sea, el segundo numerito de helloFromGPU)
- esta limitado hasta 1024, se pueden poner hasta 1024 hilos en un solo bloque


# Grillas

Veamos como trabajan los hilos...

In [ ]:
%%cu

#include <stdio.h>

/* kernel */
__global__ void Quiensoy()
{
	printf("Soy el thread (%d,%d,%d) del bloque (%d,%d,%d) [blockDim=(%d,%d,%d),gridDim=(%d,%d,%d)] \n",
        threadIdx.x,threadIdx.y,threadIdx.z,blockIdx.x,blockIdx.y,blockIdx.z,
        blockDim.x,blockDim.y,blockDim.z,gridDim.x,gridDim.y,gridDim.z);
}

int main()
{
	//TODO: pruebe distintas grillas	

	//ejemplo1: 4 blocks, y 3 threads/block: 
	dim3 nb(4,2,1); dim3 nt(3,1,1);		
	Quiensoy<<< nb, nt>>>();

	// espera a que los threads hayan terminado
	cudaDeviceSynchronize();
	return 0;
}


- nb: dimensiones de grilla en términos de bloques (x,y,z)
- nt: dimensiones de threads o hilos (x,y,z)
- dim3 es un struct de CUDA (sino ponemos nada, asume = 1)
- 8 bloques, 3 threads o bloques
- van a haber 8 x 3 = 24 impresiones
- no hay garantía de ningún orden al imprimir, puede terminar un hilo antes que otro y etc
- dice soy el 'indice del thread' del bloque 'indice del bloque'
- los indices del thread son del 0 al 2, los del bloque son en la grilla bidimensional en (x,y) correspondiente

# Suma de vectores

Hagamos primero la suma de dos vectores en la CPU, serialmente

In [ ]:
%%cu
// solucion en la cpu

#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include <ctime>

#define SIZE	1024
#define NVECES	1

void VectorAdd(int *a, int *b, int *c, int n)
{
	for(int i=0;i<n;i++)
	{
		c[i] = a[i] + b[i];
	}
}

int main(int argc, char **argv)
{
	int N;
	
	// argumentos por linea de comandos
	if(argc==2) N=atoi(argv[1]);
	else N=SIZE;

	int *a, *b, *c;

	// alocacion de memoria de host
	a = (int *)malloc(N*sizeof(int));
	b = (int *)malloc(N*sizeof(int));
	c = (int *)malloc(N*sizeof(int));

	// inicializacion
	for( int i = 0; i < N; ++i )
	{
		a[i] = i;
		b[i] = i;
		c[i] = 0;
	}


	// suma vectores c[i]=a[i]+b[i], i=0,...,N-1
	for(int i=0;i<NVECES;i++)
	VectorAdd(a, b, c, N);

	// verificacion de resultado
	for( int i = 0; i < 10; ++i){
		printf("c[%d] = %d\n", i, c[i]);
		assert(c[i]==2*i);
	}

	// liberacion de memoria
	free(a);
	free(b);
	free(c);

	return 0;
}


c[0] = 0
c[1] = 2
c[2] = 4
c[3] = 6
c[4] = 8
c[5] = 10
c[6] = 12
c[7] = 14
c[8] = 16
c[9] = 18



Ahora implementemos una suma de vectores muy simple en GPU

In [ ]:
%%cu
#include <stdio.h>
#include <assert.h>
#include <stdlib.h>

#define SIZE	1024

// kernel
__global__ void VectorAdd(int *a, int *b, int *c, int n)
{
	int i = threadIdx.x;

	if (i < n)
		c[i] = a[i] + b[i];
}

int main(int argc, char **argv)
{
	int N;
	
	if(argc==2) N=atoi(argv[1]);
	else N=SIZE;

	// punteros a memoria de host
	int *a, *b, *c;

	// punteros a memoria de device
	int *d_a, *d_b, *d_c;

	// alocacion memoria de host
	a = (int *)malloc(N*sizeof(int));
	b = (int *)malloc(N*sizeof(int));
	c = (int *)malloc(N*sizeof(int));

	// alocacion memoria de device
	cudaMalloc( &d_a, N*sizeof(int));
	cudaMalloc( &d_b, N*sizeof(int));
	cudaMalloc( &d_c, N*sizeof(int));

	// inicializacion arrays de host
	for( int i = 0; i < N; ++i )
	{
		a[i] = i;
		b[i] = i;
		c[i] = 0;
	}

	// copia de host a device
	cudaMemcpy( d_a, a, N*sizeof(int), cudaMemcpyHostToDevice );
	cudaMemcpy( d_b, b, N*sizeof(int), cudaMemcpyHostToDevice );
	cudaMemcpy( d_c, c, N*sizeof(int), cudaMemcpyHostToDevice );

	// suma paralela en el device
	VectorAdd<<< 1, N >>>(d_a, d_b, d_c, N);
	

	// copia (solo del resultado) del device a host
	cudaMemcpy( c, d_c, N*sizeof(int), cudaMemcpyDeviceToHost );

	// verificacion del resultado
	for( int i = 0; i < N; ++i){
		printf("c[%d] = %d\n", i, c[i]);
		assert(c[i]==2*i);
	}

	// liberacion memoria de host
	free(a);
	free(b);
	free(c);

	// liberacion memoria de device
	cudaFree(d_a);
	cudaFree(d_b);
	cudaFree(d_c);

	return 0;
}


01ab6b3c-a303-4897-8a5e-97d5e92b10fc.out: /tmp/tmp0hl940ad/01ab6b3c-a303-4897-8a5e-97d5e92b10fc.cu:62: int main(int, char**): Assertion `c[i]==2*i' failed.



¿Que pasa en el programa anterior si pongo N>1024 o N<1024?

Tratemos de arreglarlo para N>1024 dandole mas trabajo a cada hilo...

In [ ]:
%%cu

#include <stdio.h>
#include <assert.h>
#include <stdlib.h>
#include "/content/drive/MyDrive/Classroom/Introducción al Cálculo Numérico en Procesadores Gráficos Materia Optativa/gpu_timer.h"

#define SIZE	2048

// kernel
__global__ void VectorAdd(int *a, int *b, int *c, int n)
{
	int i = threadIdx.x;

	while(i<n){
		c[i] = a[i] + b[i];
		i+=blockDim.x;
	}
}

int main(int argc, char **argv)
{
	int N;
	
	if(argc==2) N=atoi(argv[1]);
	else N=SIZE;

	// punteros a memoria de host
	int *a, *b, *c;

	// punteros a memoria de device
	int *d_a, *d_b, *d_c;

	// alocacion memoria de host
	a = (int *)malloc(N*sizeof(int));
	b = (int *)malloc(N*sizeof(int));
	c = (int *)malloc(N*sizeof(int));

	// alocacion memoria de device
	cudaMalloc( &d_a, N*sizeof(int));
	cudaMalloc( &d_b, N*sizeof(int));
	cudaMalloc( &d_c, N*sizeof(int));

	// inicializacion arrays de host
	for( int i = 0; i < N; ++i )
	{
		a[i] = i;
		b[i] = i;
		c[i] = 0;
	}

	// copia de host a device
	cudaMemcpy( d_a, a, N*sizeof(int), cudaMemcpyHostToDevice );
	cudaMemcpy( d_b, b, N*sizeof(int), cudaMemcpyHostToDevice );
	cudaMemcpy( d_c, c, N*sizeof(int), cudaMemcpyHostToDevice );

	// timer para gpu...
	gpu_timer Reloj;
	Reloj.tic();

	// suma paralela en el device
	int nthreads_per_block=256;
	assert(nthreads_per_block<=1024);
	VectorAdd<<< 1, nthreads_per_block >>>(d_a, d_b, d_c, N);
	
	// milisegundos transcurridos
	printf("VectorAdd<<< 1, %d>>>, N= %d t= %lf ms\n", nthreads_per_block,N, Reloj.tac());	

	// copia (solo del resultado) del device a host
	cudaMemcpy( c, d_c, N*sizeof(int), cudaMemcpyDeviceToHost );

	// verificacion del resultado
	for( int i = 0; i < N; ++i){
		//printf("c[%d] = %d\n", i, c[i]);
		assert(c[i]==2*i);
	}

	// liberacion memoria de host
	free(a);
	free(b);
	free(c);

	// liberacion memoria de device
	cudaFree(d_a);
	cudaFree(d_b);
	cudaFree(d_c);

	return 0;
}


/tmp/tmp9oqokz03/3bf76a56-3bc8-4017-a25c-7dfac4a2d4ce.cu:5:10: fatal error: /content/drive/MyDrive/Classroom/Introducción al Cálculo Numérico en Procesadores Gráficos Materia Optativa/gpu_timer.h: No such file or directory
 #include "/content/drive/MyDrive/Classroom/Introducción al Cálculo Numérico en Procesadores Gráficos Materia Optativa/gpu_timer.h"
          ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
compilation terminated.



Comprobar ahora que pasa para N>1024 y mirar la performance para distintos N.

Intentemos otra solución, ahora usando muchos bloques, un hilo por bloque...

In [ ]:
%%cu
// solucion paralela parchada, correcta pero ineficiente...
// nvcc suma_vectores_gpu_muchos_bloques_un_thread_por_bloque.cu -arch=sm_30
// si no usamos sm_30 da error a 65536 porque compila para abajo!
#include <stdio.h>
#include <assert.h>
#include <stdlib.h>
#include "/content/drive/MyDrive/Classroom/Introducción al Cálculo Numérico en Procesadores Gráficos Materia Optativa/gpu_timer.h"

#define SIZE	1024

// kernel
__global__ void VectorAdd(int *a, int *b, int *c, int n)
{
	int i = blockIdx.x;

	if(i<n){
		c[i] = a[i] + b[i];
	}
}

int main(int argc, char **argv)
{
	int N;
	
	if(argc==2) N=atoi(argv[1]);
	else N=SIZE;

	// punteros a memoria de host
	int *a, *b, *c;

	// punteros a memoria de device
	int *d_a, *d_b, *d_c;

	// alocacion memoria de host
	a = (int *)malloc(N*sizeof(int));
	b = (int *)malloc(N*sizeof(int));
	c = (int *)malloc(N*sizeof(int));

	// alocacion memoria de device
	cudaMalloc( &d_a, N*sizeof(int));
	cudaMalloc( &d_b, N*sizeof(int));
	cudaMalloc( &d_c, N*sizeof(int));

	// inicializacion arrays de host
	for( int i = 0; i < N; ++i )
	{
		a[i] = i;
		b[i] = i;
		c[i] = 0;
	}

	// copia de host a device
	cudaMemcpy( d_a, a, N*sizeof(int), cudaMemcpyHostToDevice );
	cudaMemcpy( d_b, b, N*sizeof(int), cudaMemcpyHostToDevice );
	cudaMemcpy( d_c, c, N*sizeof(int), cudaMemcpyHostToDevice );

	// timer para gpu...
	gpu_timer Reloj;
	Reloj.tic();

	// suma paralela en el device
	int nbloques=N;
	assert(nbloques<2147483647);
	VectorAdd<<< N, 1 >>>(d_a, d_b, d_c, N);
	
	// milisegundos transcurridos
	printf("VectorAdd<<< N, 1 >>>, N= %d t= %lf ms\n", N, Reloj.tac());	

	// copia (solo del resultado) del device a host
	cudaMemcpy( c, d_c, N*sizeof(int), cudaMemcpyDeviceToHost );

	// verificacion del resultado
	for( int i = 0; i < N; ++i){
		//printf("c[%d] = %d\n", i, c[i]);
		assert(c[i]==2*i);
	}

	// liberacion memoria de host
	free(a);
	free(b);
	free(c);

	// liberacion memoria de device
	cudaFree(d_a);
	cudaFree(d_b);
	cudaFree(d_c);

	return 0;
}


/tmp/tmp52nmpjdu/96632609-2e98-47b7-8211-2d1d7a382d41.cu:7:10: fatal error: /content/drive/MyDrive/Classroom/Introducción al Cálculo Numérico en Procesadores Gráficos Materia Optativa/gpu_timer.h: No such file or directory
 #include "/content/drive/MyDrive/Classroom/Introducción al Cálculo Numérico en Procesadores Gráficos Materia Optativa/gpu_timer.h"
          ^~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
compilation terminated.



Probar la performance con distintos N.

Y ahora, la solucion mas general, muchos bloques, muchos hilos por bloque

In [ ]:
%%cu
// solucion paralela optima...

#include <stdio.h>
#include <assert.h>
#include <stdlib.h>

#include "/content/drive/MyDrive/Classroom/Introducción al Cálculo Numérico en Procesadores Gráficos Materia Optativa/gpu_timer.h"

#define SIZE	1024

// kernel
__global__ void VectorAdd(int *a, int *b, int *c, int n)
{
	// indice de thread mapeado a indice de array 
	int i = blockIdx.x * blockDim.x + threadIdx.x;
	if (i < n)
		c[i] = a[i] + b[i];
}

int main(int argc, char **argv)
{
	int N;
	
	if(argc==2) N=atoi(argv[1]);
	else N=SIZE;

	// punteros a memoria de host
	int *a, *b, *c;

	// punteros a memoria de device
	int *d_a, *d_b, *d_c;

	// alocacion memoria de host
	a = (int *)malloc(N*sizeof(int));
	b = (int *)malloc(N*sizeof(int));
	c = (int *)malloc(N*sizeof(int));

	// alocacion memoria de device
	cudaMalloc( &d_a, N*sizeof(int));
	cudaMalloc( &d_b, N*sizeof(int));
	cudaMalloc( &d_c, N*sizeof(int));

	// inicializacion arrays de host
	for( int i = 0; i < N; ++i )
	{
		a[i] = i;
		b[i] = i;
		c[i] = 0;
	}

	// copia de host a device
	cudaMemcpy( d_a, a, N*sizeof(int), cudaMemcpyHostToDevice );
	cudaMemcpy( d_b, b, N*sizeof(int), cudaMemcpyHostToDevice );
	cudaMemcpy( d_c, c, N*sizeof(int), cudaMemcpyHostToDevice );

	// timer para gpu...
	gpu_timer Reloj;
	Reloj.tic();

	// grilla de threads suficientemente grande...
	dim3 nThreads(256);
	dim3 nBlocks((N + nThreads.x - 1) / nThreads.x);
	// suma paralela en el device
	VectorAdd<<< nBlocks, nThreads >>>(d_a, d_b, d_c, N);
	
	// milisegundos transcurridos
	printf("VectorAdd<<< %d, %d >>>, N= %d t= %lf ms\n", nBlocks.x, nThreads.x, N, Reloj.tac());	

	// copia (solo del resultado) del device a host
	cudaMemcpy( c, d_c, N*sizeof(int), cudaMemcpyDeviceToHost );

	// verificacion del resultado
	for( int i = 0; i < N; ++i){
		//printf("c[%d] = %d\n", i, c[i]);
		assert(c[i]==2*i);
	}

	// liberacion memoria de host
	free(a);
	free(b);
	free(c);

	// liberacion memoria de device
	cudaFree(d_a);
	cudaFree(d_b);
	cudaFree(d_c);

	return 0;
}


**Tarea:** Comparar la performance de los tres codigos para GPU, con el de CPU, para distintos valores de N.

Probemos ahora usar la libreria Thrust, que veremos mas adelante, para simplificar la programacion

In [ ]:
%%cu
// solucion thrust

#include <stdio.h>
#include <assert.h>
#include <stdlib.h>
#include "/content/drive/MyDrive/Classroom/Introducción al Cálculo Numérico en Procesadores Gráficos Materia Optativa/gpu_timer.h"

#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/sequence.h>
#include <stdlib.h>

#define SIZE	1024

// no hay kernel!...

int main(int argc, char **argv)
{
	int N;
	
	if(argc==2) N=atoi(argv[1]);
	else N=SIZE;

	// vectores (containers) de device
	thrust::device_vector<int> d_a(N);
	thrust::device_vector<int> d_b(N);
	thrust::device_vector<int> d_c(N);

	// inicializacion arrays de device
	thrust::sequence(d_a.begin(),d_a.end());
	thrust::copy(d_a.begin(),d_a.end(),d_b.begin());

	// timer para gpu...
	gpu_timer Reloj;
	Reloj.tic();

	// suma paralela en el device
	using namespace thrust::placeholders;
	thrust::transform(d_a.begin(), d_a.end(), d_b.begin(), d_c.begin(), _1+_2);	
	
	// milisegundos transcurridos
	printf("thrust::transform, N= %d t= %lf ms\n", N, Reloj.tac());	

	// crea un vector de host y copia el resultado del device
	thrust::host_vector<int> c(d_c);

	// verificacion del resultado
	for( int i = 0; i < N; ++i){
		//printf("c[%d] = %d\n", i, c[i]);
		assert(c[i]==2*i);
	}

	//la memoria se libera automaticamente...

	return 0;
}
